In [86]:
import markovify
from tqdm import tqdm
from random import randint
from sklearn.model_selection import train_test_split
import evaluate

In [87]:
with open("./voina-i-mir.txt", "r") as file:
    text = file.read()

data = markovify.Text(text).parsed_sentences

train, test = train_test_split(data, train_size=0.9)
train_text = " ".join([" ".join(sts) for sts in train])

print(f"Length of traingin text: {len(train_text)}")   
print(f"Sentences:\n    In train: {len(train)}\n    In test:  {len(test)}")

models = [None, None]
MIN_SIZE = 2
MAX_SIZE = 5

for size in tqdm(range(MIN_SIZE, MAX_SIZE + 1)):
    models.append(markovify.Text(train_text, state_size=size))


Length of traingin text: 1352738
Sentences:
    In train: 29118
    In test:  3236


100%|██████████| 4/4 [00:01<00:00,  2.04it/s]


In [98]:
MAX_GENERATIONS_ATTEMPTS = 10

results = [[], []]
references = [[], []]

bleu = evaluate.load('bleu')

for size in range(MIN_SIZE, MAX_SIZE + 1):
    print(f"====================== Chain size: {size} ======================")
    results.append([])
    references.append([])
    for ref in tqdm(test):
        if len(ref) < 2:
            continue
        for _ in range(MAX_GENERATIONS_ATTEMPTS):
            try:
                sentence = models[size].make_sentence_with_start(f"{ref[0]} {ref[1]}")
                if sentence:
                    results[size].append(sentence)
                    references[size].append([" ".join(ref)])
                break
            except:
                continue
    
    score = bleu.compute(predictions=results[-1], references=references[-1])        
    
    print(f"Successful generations: {len(results[-1])}")
    print(f"BLEU: {score['bleu']}")
            

====================== Chain size: 2 ======================


100%|██████████| 3236/3236 [00:04<00:00, 663.35it/s]


Successful generations: 1584
BLEU: 0.08104972407624683
====================== Chain size: 3 ======================


100%|██████████| 3236/3236 [00:12<00:00, 250.12it/s]


Successful generations: 910
BLEU: 0.09779562976200402
====================== Chain size: 4 ======================


100%|██████████| 3236/3236 [00:20<00:00, 160.84it/s]


Successful generations: 362
BLEU: 0.11312797335851592
====================== Chain size: 5 ======================


100%|██████████| 3236/3236 [00:22<00:00, 141.98it/s]

Successful generations: 69
BLEU: 0.11645654374010592


In [99]:
model_size = 5

for result, reference in zip(results[model_size], references[model_size]):
    print("===================================================================================================")
    print(f"Reference: {' '.join(reference)}\nGenerated: {result}")

Reference: 68 г.: или ты Стр.
Generated: 68 г.: мы дальше Стр. — Позвольте мне спросить, — сказал он, — вы знаете, что я люблю вас, но...
Reference: Вместо: а умру — в I и II изд.
Generated: Вместо: а то бывает... — в I и II изд. — Ай да Nicolas!
Reference: Он не только не интересовался ими, но он и не заметил, а тотчас же забыл их.
Generated: Он не оглядывался и глядя в лицо ребенка, всё слушал — в I и II изд. — Oh, oh!
Reference: 69 г.: Македонского, и что это хорошо.
Generated: 69 г.: так как нет на свете положения, — в изд.
Reference: После слов: кроме обуви. — в Р.
Generated: После слов: племянниц графа, — в Р. — Нет, — сказала княжна Марья.
Reference: Вместо: — Знайте, — в I и II изд.
Generated: Вместо: — Ей Богу, не знаю, куда я его дел, — сказал он.
Reference: После слов: далеким воспоминанием. — в Р.
Generated: После слов: на Лаврушку, — в I изд. — Хорошо, хорошо, — сказал полковой командир и обратился к майору Экономову.
Reference: После слов: покачал головой. — в Р.
Generate